In [1]:
import cv2
import numpy as np


In [2]:
# Load YOLO object detection model (pre-trained)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [3]:
# Load class labels for YOLO (Coco dataset)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize video capture from video file or camera
video_path = "railway_station.mp4.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

In [4]:
crowd_threshold = 10

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or cannot read frame.")
        break
    
    # Get the frame height and width
    height, width, channels = frame.shape

    # Prepare the image for YOLO (blob format)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Forward pass through the network
    outputs = net.forward(output_layers)

    # Data containers for detected class ids, confidences, and boxes
    class_ids = []
    confidences = []
    boxes = []

    # Iterate through the YOLO output
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter for 'person' class and threshold confidence level (e.g., 0.5)
            if classes[class_id] == "person" and confidence > 0.5:
                # Object detected, get bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate top-left corner of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Append to the list of detected boxes, confidences, and class_ids
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression to remove redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Count the number of people detected
    crowd_count = 0

    # Draw bounding boxes for detected persons
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))

            # Draw rectangle and label on the video frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {confidence}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

            # If the label is 'person', increment crowd count
            if label == "person":
                crowd_count += 1

    # Display crowd count on the video frame
    cv2.putText(frame, f"Crowd Count: {crowd_count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

    # Check if the crowd count exceeds the predefined threshold and trigger an alert
    if crowd_count > crowd_threshold:
        cv2.putText(frame, "ALERT: Crowd size exceeded!", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
        print("ALERT: Crowd size exceeded!")

    # Display the frame with detections
    cv2.imshow("Railway Station - Crowd Detection", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!
ALERT: Crowd size exceeded!


In [5]:
# Release video capture and close display windows
cap.release()
cv2.destroyAllWindows()